In [42]:
from collections import defaultdict
import re
import os

def extract_timing_summary(filepath, table_amount):
    with open(filepath, 'r') as file:
        lines = file.readlines()

    in_summary = False
    summary = {}
    count = 0 
    for line in lines:
        if count > table_amount:
            break #We have read the defined amount of tables
        if '--- Timing Summary ---' in line:
            in_summary = True
            continue #skip rest of loop
        if in_summary:
            if line.strip().startswith('---'):
                in_summary = False
                count += 1 
                continue  # End of timing summary
            # Match lines like "Label: value unit"
            match = re.match(r'(.+?):\s+([\d.]+)\s*(ms|s|us)?', line)
            if match:
                label = match.group(1).strip() + " " + str(count)
                value = float(match.group(2))
                unit = match.group(3) or 'ms'  # default to ms if missing
                summary[label] = {'value': value, 'unit': unit}

    return summary

def extract_all_timing_summaries(filepath):
    with open(filepath, 'r', errors='ignore') as file:
        lines = file.readlines()

    summaries = []
    in_summary = False
    current_summary = {}

    for line in lines:
        if '--- Timing Summary ---' in line:
            if current_summary:
                summaries.append(current_summary)
                current_summary = {}
            in_summary = True
            continue

        if in_summary:
            if line.strip() == '' or line.strip().startswith('---'):
                if current_summary:
                    summaries.append(current_summary)
                    current_summary = {}
                in_summary = False
                continue

            match = re.match(r'(.+?):\s+([\d.]+)\s*(ms|s|us)?', line)
            if match:
                label = match.group(1).strip()
                value = float(match.group(2))
                unit = match.group(3) or 'ms'
                current_summary[label] = {'value': value, 'unit': unit}

    # Catch the last block if file ends right after it
    if current_summary:
        summaries.append(current_summary)

    return summaries
def average_timing_summaries(summaries):
    totals = defaultdict(float)
    counts = defaultdict(int)

    for summary in summaries:
        for key, value in summary.items():
            totals[key] += value['value']
            counts[key] += 1

    averages = {key: totals[key] / counts[key] for key in totals}
    return averages

cwd = os.getcwd()

path = cwd + "/GPU/out/out_gpu.txt"
path_literal = cwd + "/GPU/out/out_gpu_literal.txt"

summaries = extract_all_timing_summaries(path)

def print_average_summary(summaries):
    if not summaries:
        print("No timing summaries found.")
    else:
        averages = average_timing_summaries(summaries)
        print("\n--- Averaged Timing Summary (over {} runs) ---".format(len(summaries)))
        for label, avg in sorted(averages.items()):
            print(f"{label}: {avg:.3f} ms")

# print_average_summary(summaries)
# if not summaries:
#     print("No timing summaries found.")
# else:
#     averages = average_timing_summaries(summaries)
#     print("\n--- Averaged Timing Summary (over {} runs) ---".format(len(summaries)))
#     for label, avg in sorted(averages.items()):
#         print(f"{label}: {avg:.3f} ms")
        
summaries_literal = extract_all_timing_summaries(path_literal)
if not summaries_literal:
    print("No timing summaries found for literals")
else:
    averages = average_timing_summaries(summaries_literal)
    print("\n--- Averaged Timing Summary for literals (over {} runs) ---".format(len(summaries_literal)))
    for label, avg in sorted(averages.items()):
        print(f"{label}: {avg:.3f} ms")
    

        


--- Averaged Timing Summary for literals (over 10 runs) ---
Data transfer to GPU: 11.000 ms
Device memory alloc: 309.200 ms
Host preprocessing: 4112.200 ms
Kernel execution: 912.397 ms
Post GPU processing: 0.000 ms
Regex pattern matching: 36.400 ms
Total runtime: 5346.200 ms


In [49]:
def print_average_summary(path, comment = ''):
    summaries = extract_all_timing_summaries(path)
    if not summaries:
        print("No timing summaries found.")
    else:
        averages = average_timing_summaries(summaries)
        print("\n--- Averaged Timing Summary (over {} runs) ---".format(len(summaries)), comment)
        for label, avg in sorted(averages.items()):
            print(f"{label}: {avg:.3f} ms")

cwd = os.getcwd()
path = cwd + "/GPU/out/"
path_literal_10mb_small = path + 'out_gpu_literal_10_mb_small.txt'
path_mixed_10mb_small = path + 'out_gpu_mixed_10mb_small.txt'
path_mixed_million_small = path + 'out_gpu_mixed_million_small.txt'
path_literal_million_small = path + 'out_gpu_literal_million_small.txt'
print_average_summary(path_literal_10mb_small, 'literal 10mb small')
print_average_summary(path_mixed_10mb_small,  'mixed 10mb small')
print_average_summary(path_literal_million_small, 'literal million small')
print_average_summary(path_mixed_million_small,  'mixed million small')


--- Averaged Timing Summary (over 5 runs) --- literal 10mb small
Data transfer to GPU: 2.000 ms
Device memory alloc: 207.800 ms
Host preprocessing: 539.000 ms
Kernel execution: 59.076 ms
Post GPU processing: 0.000 ms
Regex pattern matching: 28.600 ms
Total runtime: 809.200 ms

--- Averaged Timing Summary (over 5 runs) --- mixed 10mb small
Data transfer to GPU: 1.800 ms
Device memory alloc: 232.600 ms
Host preprocessing: 539.800 ms
Kernel execution: 150592.219 ms
Post GPU processing: 0.000 ms
Regex pattern matching: 40.400 ms
Total runtime: 173058.600 ms

--- Averaged Timing Summary (over 3 runs) --- literal million small
Data transfer to GPU: 10.000 ms
Device memory alloc: 295.333 ms
Host preprocessing: 3968.333 ms
Kernel execution: 293.732 ms
Post GPU processing: 0.000 ms
Regex pattern matching: 9.333 ms
Total runtime: 4568.667 ms

--- Averaged Timing Summary (over 3 runs) --- mixed million small
Data transfer to GPU: 10.000 ms
Device memory alloc: 302.333 ms
Host preprocessing: 4066